In [2]:
from nltk.tokenize import sent_tokenize
from sklearn.model_selection import train_test_split
from nltk import ngrams

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\91875\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Import file and preprocessing

In [4]:
text=open('speeches.txt','rt',encoding='utf-8')

In [5]:
fin=text.read()

In [6]:
fin=fin.replace('\n', '')
fin=fin.replace('.','. ')

In [7]:
sen=sent_tokenize(fin)
text.close()

In [8]:
final=[]
for i in sen:
    final.append('<s> '+i.replace('.','').replace('?','').replace(',','').replace(' - ','')+' </s>')

In [9]:
xTrain, xTest = train_test_split(final, test_size = 0.2, random_state = 0)

# n-grams listing and MLEs

In [10]:
big=dict()
n = 2
bc=0
for sen in xTrain:
    bigrams = ngrams(sen.split(), n)
    for x in bigrams:
        if(x not in big):
            big[x]=1
        else:
            big[x]+=1
        bc=bc+1       

In [11]:
tri=dict()
n = 3
tc=0
for sen in xTrain:
    trigrams = ngrams(sen.split(), n)
    for x in trigrams:
        if(x not in tri):
            tri[x]=1
        else:
            tri[x]=tri[x]+1
        tc=tc+1    

In [12]:
uni=dict()
n = 1
uc=0
for sen in xTrain:
    unigrams = ngrams(sen.split(), n)
    for x in unigrams:
        if(x[-1]=='.'):
            x=x[:-2]
        if(x not in uni):
            uni[x]=1
        else:
            uni[x]=uni[x]+1
        uc=uc+1    


In [13]:
quad=dict()
n = 4
qc=0
for sen in xTrain:
    quadgrams = ngrams(sen.split(), n)
    for x in quadgrams:
        if(x not in quad):
            quad[x]=1
        else:
            quad[x]=quad[x]+1
        qc=qc+1    

In [14]:
uc,bc,tc,qc

(162132, 148583, 135034, 121899)

In [15]:
len(uni),len(big),len(tri),len(quad)

(8043, 50617, 90373, 103141)

In [16]:
MLEB=dict()
for i in big:
    MLEB[i]=big[i]/uni[i[:-1]]

In [17]:
MLET=dict()
for i in tri:
    MLET[i]=tri[i]/big[i[:-1]]

In [18]:
MLEQ=dict()
for i in quad:
    MLEQ[i]=quad[i]/tri[i[:-1]]

In [19]:
MLEU=dict()
for i in uni:
    MLEU[i]=uni[i]/uc

In [20]:
import numpy as np

# n-gram Generator

In [21]:
def predict_first(gram,n):
    x=['<s>']
    b=[]
    c=[]
    for i in range(n-2):
        x=list(x)
        keys, values = zip(*gram[i+1].items()) 
        values=list(values)
        keys=list(keys)
        for j in keys:
            l=list(j)
            if(l[:i+1]==x):
                c.append(gram[i+1][j])
                b.append(j)
#         print(b,c)
        array=np.random.multinomial(1 , c , size=1)
        for k in range(len(array[0])):
            if(array[0][k]==1):
#                 print(str(b[k]),end=' ')
                x=b[k]
        b=[]
        c=[]
    return(list(x))
def generator(gram,n,m):
#     keys, values = zip(*gram[n-2].items()) 
#     values=list(values)
#     x='<s>'
#     b=[]
#     c=[]
#     for j in keys:
#         if(j[0]==x):
#             c.append(gram[n-2][j])
#             b.append(j)
#     c=np.array(c)
#     c=c/sum(c)        
#     array=np.random.multinomial(1 , c , size=1)
#     for k in range(len(array[0])):
#         if(array[0][k]==1):
#             print(str(b[k][:n-1]),end=' ')
#             x=b[k] 
    if(n==1):
        keys, values = zip(*MLEU.items()) 
        values=list(values)
        array=np.random.multinomial(1 , values , size=m)
        print('<s>',end=' ')
        for i in array:
            for j in range(len(keys)):
                if(i[j]==1):
                    if(keys[j][0]=='</s>'):
                        print(str(keys[j][0]),end=' ')
                        return
                    print(str(keys[j][0]),end=' ')
        return
    x=predict_first(gram,n)
    b=[]
    c=[]
    keys, values = zip(*gram[n-1].items()) 
    values=list(values)
    keys=list(keys)
    print(*x,end=' ')
    for i in range(m):
        s=0
        for j in keys:
            l=list(j)
#             print(l[:n-1],x)
            if(l[:n-1]==x):
                c.append(gram[n-1][j])
                b.append(j)
#                 print(j,x,"Match")
                s=1
        if(s==0):
            break
        array=np.random.multinomial(1 , c , size=1)
        for k in range(len(array[0])):
            if(array[0][k]==1):
                print((list(b[k]))[-1],end=' ')
                x=b[k][1:]
        x=list(x)
        c=[]
        b=[]    

In [22]:
MLE=[MLEU,MLEB,MLET,MLEQ]
gram=[uni,big,tri,quad]
n=[uc,bc,tc,qc]
print("Unigram")
generator(MLE,1,100)
print()
print("Bigram")
generator(MLE,2,100)
print()
print("Trigram")
generator(MLE,3,100)
print()
print("Quadgram")
generator(MLE,4,100)
print()

Unigram
<s> on not We’re We’re </s> 
Bigram
<s> When the magazines no lobbyist asked every other names of people </s> 
Trigram
<s> Why do we have that same thing I thought I’d do and go directly against whoever they happen to believe </s> 
Quadgram
<s> He’s doing a tremendous job </s> 


# Perplexity for classical approach

In [23]:
from math import log as log
import math

In [24]:
def perplexity(gram,MLE,xTest,n):
    ans=0
    l=0
    for i in xTest:
        ngram = ngrams(sen.split(), n)
        for x in ngram:
            if(x not in gram[n-1]):
                ans+=0
                l+=len(x)
            else:
                ans+=log(1/gram[n-1][x],2)
                l+=len(x)
    return(math.pow(2,(-1*ans/l)))
                

In [25]:
print("For unigram ",perplexity(gram,MLE,xTest,1))
print("For bigram ",perplexity(gram,MLE,xTest,2))
print("For trigram ",perplexity(gram,MLE,xTest,3))
print("For quadgram ",perplexity(gram,MLE,xTest,4))

For unigram  658.2817108218651
For bigram  3.318241956119028
For trigram  1.0958726911352492
For quadgram  1.0


# Neural approach

In [43]:
from keras.layers import SimpleRNN
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout,SimpleRNN 
from keras.preprocessing.sequence import pad_sequences


# Vanilla RNN

In [189]:
a = list()
n = 3
for i in xTrain:
  grams = ngrams(i.split(),3)
  for gram in grams:
        a.append(list(gram))
    
t = Tokenizer()
t.fit_on_texts(a)
b = t.texts_to_sequences(a)

length = len(t.word_index) + 1

b = np.array(b)

x_train = b[:,:-1]
y_train = b[:,-1]

y_train = to_categorical(y_train, num_classes=length)

model1 = Sequential()
model1.add(Embedding(length, 100, input_length=n-1))
model1.add(Dropout(0.15))
model1.add(SimpleRNN(50))
model1.add(Dense(75, activation='relu'))
model1.add(Dense(length, activation='softmax'))

model1.summary()

model1.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
history_rnn = model1.fit(x_train, y_train, batch_size=128, epochs=10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 2, 100)            652900    
_________________________________________________________________
dropout_5 (Dropout)          (None, 2, 100)            0         
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 50)                7550      
_________________________________________________________________
dense_27 (Dense)             (None, 75)                3825      
_________________________________________________________________
dense_28 (Dense)             (None, 6529)              496204    
Total params: 1,160,479
Trainable params: 1,160,479
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
135034/135034 [==============================] - 42s 314us/step - loss: 5.7881 - acc: 0.1312
Epoch 2/10
135034/135034 [==

# LSTM

In [188]:
model2 = Sequential()
model2.add(Embedding(length, 100, input_length=n-1))
model2.add(LSTM(50, return_sequences=True))
model2.add(LSTM(50))
model2.add(Dense(50, activation='relu'))
model2.add(Dense(length, activation='softmax'))

model2.summary()
model2.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
history_lstm = model2.fit(x_train, y_train, batch_size=128, epochs=10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 2, 100)            652900    
_________________________________________________________________
lstm_22 (LSTM)               (None, 2, 50)             30200     
_________________________________________________________________
lstm_23 (LSTM)               (None, 50)                20200     
_________________________________________________________________
dense_25 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_26 (Dense)             (None, 6529)              332979    
Total params: 1,038,829
Trainable params: 1,038,829
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
135034/135034 [==============================] - 58s 431us/step - loss: 6.0992 - acc: 0.0985
Epoch 2/10
135034/135034 [==

# Generators for RNN and LSTM

In [190]:
#LSTM
import random
for i in range(5):
  result = list()
  text = xTrain[random.randint(0,len(xTrain))].split(" ")
  f = 0
  if(len(text)>=n):
    result = text[:n-1]
    l = n
    while(result[-1]!="</s>"):
      encoded = t.texts_to_sequences(result)
      encoded=[i[0] for i in encoded]
      encoded = pad_sequences([encoded], maxlen= 2, padding='pre')
      encoded = np.array(encoded)
      yhat = model2.predict_classes(encoded, verbose=0)
      ans = ''
      for word, index in t.word_index.items():
        if index == yhat:
          ans = word
          break
      result.append(ans)
      l+=1
    print(" ".join(result))

<s> We’re going to be a lot of people </s>
<s> They have to be a lot of people </s>
<s> I’ll have to be a lot of people </s>
<s> No </s>
<s> It’s a lot of people </s>


In [191]:
#RNN
import random
for i in range(5):
  result = list()
  text = xTrain[random.randint(0,len(xTrain))].split(" ")
  f = 0
  if(len(text)>=n):
    result = text[:n-1]
    l = n
    while(result[-1]!="</s>"):
      encoded = t.texts_to_sequences(result)
      encoded=[i[0] for i in encoded]
      encoded = pad_sequences([encoded], maxlen= 2, padding='pre')
      encoded = np.array(encoded)
      yhat = model1.predict_classes(encoded, verbose=0)
      ans = ''
      for word, index in t.word_index.items():
        if index == yhat:
          ans = word
          break
      result.append(ans)
      l+=1
    print(" ".join(result))

<s> THAT'S going to be a lot of people </s>
<s> And i don’t know </s>
<s> In the world </s>
<s> I think it’s going to be a lot of people </s>
<s> But i don’t know </s>


Neural networks essentially learns the patterns of the language while in n-grams approach we use probability to generate the most fitting word to a set of words. LSTM here gives a lower accuracy than RNN but it could be trained in a better way than this. Neural networks will give a better result than classical approaches.